# 딥 러닝을 이용한 자연어 처리 입문

아래 링크의 E-book을 보고 실습한 내용입니다.

WikiDocs 주소: https://wikidocs.net/31766

# 6장 토픽 모델링

## 2절 잠재 디리클레 할당 (Latent Dirichlet Allocation, LDA)


## 뉴스 그룹 데이터를 활용한 LDA 실습 - gensim

### 텍스트 전처리


In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from nltk.corpus import stopwords

dataset = fetch_20newsgroups(
    shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data


news_df = pd.DataFrame({"document": documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace(
    r"[^a-zA-Z]", " ", regex=True)
# 길이가 짧은 단어 제거 및 단어 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(
    lambda x: ' '.join([w.lower() for w in x.split() if len(w) > 3]))

# 불용어 제거
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(
    lambda x: [item for item in x if item not in stop_words])

tokenized_doc


0        [well, sure, story, seem, biased, disagree, st...
1        [yeah, expect, people, read, actually, accept,...
2        [although, realize, principle, strongest, poin...
3        [notwithstanding, legitimate, fuss, proposal, ...
4        [well, change, scoring, playoff, pool, unfortu...
                               ...                        
11309    [danny, rubenstein, israeli, journalist, speak...
11310                                                   []
11311    [agree, home, runs, clemens, always, memorable...
11312    [used, deskjet, orange, micros, grappler, syst...
11313    [argument, murphy, scared, hell, came, last, y...
Name: clean_doc, Length: 11314, dtype: object

In [4]:
from gensim import corpora


# Vocabulary 생성
dictionary = corpora.Dictionary(tokenized_doc)
# 정수 인코딩 및 빈도수 기록
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]

print("(word_id, 빈도수), 뉴스 2의 결과:", corpus[1])

(word_id, 빈도수), 뉴스 2의 결과: [(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [5]:
print("word_id가 66인 단어를 dictionary에서 찾기:", dictionary[66])
print("전체 단어:", len(dictionary))

word_id가 66인 단어를 dictionary에서 찾기: faith
전체 단어: 64281


In [6]:
import gensim

# LDA 학습을 위해 토픽의 개수를 지정
NUM_TOPICS = 20

# LDA 모델 객체 생성
# - passes: 알고리즘 동작 횟수, 적절히 수렴할 수 있도록 적당한 값 할당
ldamodel = gensim.models.ldamodel.LdaModel(
    corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

# 토픽 목록 출력
# - num_words: 총 몇 개의 단어를 출력할 것인지
topics = ldamodel.print_topics(num_words=4)
print("각 토픽별 단어 분포")
print("(토픽ID, 중요도*단어, ... 중요도*단어")
for topic in topics:
    print(topic)


각 토픽별 단어 분포
(토픽ID, 중요도*단어, ... 중요도*단어
(0, '0.007*"many" + 0.007*"science" + 0.005*"books" + 0.004*"book"')
(1, '0.011*"windows" + 0.010*"system" + 0.008*"drive" + 0.007*"like"')
(2, '0.016*"period" + 0.011*"play" + 0.010*"went" + 0.010*"power"')
(3, '0.017*"would" + 0.015*"people" + 0.009*"know" + 0.008*"think"')
(4, '0.027*"space" + 0.009*"nasa" + 0.009*"program" + 0.008*"center"')
(5, '0.033*"scsi" + 0.014*"nist" + 0.009*"ncsl" + 0.008*"swap"')
(6, '0.023*"mail" + 0.018*"please" + 0.014*"send" + 0.013*"list"')
(7, '0.052*"israel" + 0.032*"israeli" + 0.020*"arab" + 0.011*"palestinian"')
(8, '0.015*"myers" + 0.012*"cover" + 0.009*"cubs" + 0.009*"openwindows"')
(9, '0.014*"public" + 0.014*"encryption" + 0.011*"security" + 0.011*"government"')
(10, '0.016*"ripem" + 0.014*"pain" + 0.012*"insurance" + 0.009*"road"')
(11, '0.014*"pitt" + 0.014*"banks" + 0.013*"gordon" + 0.013*"surrender"')
(12, '0.012*"like" + 0.012*"would" + 0.011*"good" + 0.008*"know"')
(13, '0.016*"year" + 0.013*"game" +

### pyLDAvis 결과 해설

- 각 원은 토픽들을 나타낸다. (20개)
- 원과 원 사이의 거리는 각 토픽들이 얼마나 다른지 보여준다. (가까우면 유사한 토픽)

- 주의
  - LDA 결과에서는 토픽번호가 0번부터 인덱싱이 시작
  - LDA 시각화 결과에서는 토픽번호가 1번부터 인덱싱이 시작


In [7]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)


C:\Users\HTJ\.conda\envs\tf\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [8]:
print("문서별 토픽 분포")
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(f'{i}번째 문서의 토픽 분포:{topic_list}')

문서별 토픽 분포
0번째 문서의 토픽 분포:[(1, 0.16368216), (3, 0.2726759), (7, 0.14757475), (15, 0.017892573), (16, 0.21715955), (19, 0.1694644)]
1번째 문서의 토픽 분포:[(1, 0.26939818), (3, 0.20631717), (11, 0.029321698), (13, 0.091399446), (19, 0.38379943)]
2번째 문서의 토픽 분포:[(3, 0.63415664), (4, 0.023003893), (7, 0.29005828), (12, 0.039867517)]
3번째 문서의 토픽 분포:[(0, 0.020107511), (1, 0.21195239), (3, 0.3276723), (4, 0.024833484), (9, 0.2704295), (10, 0.08397533), (12, 0.0508046)]
4번째 문서의 토픽 분포:[(2, 0.07595477), (6, 0.034360953), (13, 0.5538153), (18, 0.3062259)]


In [13]:
# Dataframe으로 보기 쉽게 정리

df = pd.DataFrame()
for i, topic_list in enumerate(ldamodel[corpus]):
    doc = topic_list[0] if ldamodel.per_word_topics else topic_list
    doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
    for j, (topic_id, prop_topic) in enumerate(doc):
        if j == 0:
            df = df.append(pd.Series([int(topic_id), round(
                prop_topic, 4), topic_list]), ignore_index=True)
        else:
            break


In [32]:
topic_table = df
topic_table.columns = ["1등 토픽", "비중", "나머지 토픽들의 비중"]
topic_table["1등 토픽"] = topic_table["1등 토픽"].astype(int)
topic_table[:10]


,1등 토픽,비중,나머지 토픽들의 비중
0,3,0.2727,"[(1, 0.1636818), (3, 0.2727023), (7, 0.1475734..."
1,19,0.3839,"[(1, 0.26941252), (3, 0.20614092), (11, 0.0293..."
2,3,0.6341,"[(3, 0.63407326), (4, 0.023000062), (7, 0.2900..."
3,3,0.3281,"[(0, 0.020106925), (1, 0.21197617), (3, 0.3280..."
4,13,0.5537,"[(2, 0.07599514), (6, 0.034394607), (13, 0.553..."
5,19,0.8497,"[(8, 0.10932207), (19, 0.849743)]"
6,12,0.4656,"[(1, 0.37525398), (7, 0.01492148), (10, 0.0149..."
7,3,0.6870,"[(3, 0.68700653), (4, 0.07027394), (7, 0.06222..."
8,17,0.4553,"[(0, 0.10233047), (3, 0.41739187), (17, 0.4552..."
9,12,0.6108,"[(1, 0.02515611), (3, 0.17504744), (8, 0.02906..."
